In [70]:
#importing all the libraries

In [2]:
import sys
assert sys.version_info >= (3, 5)

In [3]:
import sklearn
assert sklearn.__version__ >= "0.20"

In [4]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"
from numba import cuda

In [5]:
import numpy as np
import os

In [6]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11364528037206672160
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14028889225327616193
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 13712867945323501562
physical_device_desc: "device: XLA_GPU device"
]


In [7]:
len(cuda.gpus)
cuda.gpus[0].name


b'GeForce RTX 2070 SUPER'

In [8]:
tf.config.experimental.list_physical_devices('GPU')

[]

In [11]:
print(os.getcwd())

C:\Users\Dominik\python course


In [53]:
#preprocessing

In [28]:
text = open("TheSunandHerFlowers.txt").read().lower()

In [29]:
print(text[:312])


how is it so easy for you
to be kind to people he asked

milk and honey dripped 
from my lips as i answered

cause people have not 
been kind to me


the first boy that kissed me 
held my shoulders down 
like the handlebars of
the first bicycle 
he ever rode
i was five

he had the smell of 
starvation on his li


In [30]:
"".join(sorted(set(text.lower())))

'\t\n\x0c ()*,-.0123456789:abcdefghijklmnopqrstuvwxyz|â“”€™'

In [31]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)

In [54]:
#checking whether tokenizer worked

In [32]:
tokenizer.texts_to_sequences(["First"])

[[18, 5, 11, 9, 3]]

In [33]:
tokenizer.sequences_to_texts([[18, 5, 11, 9, 3]])


['f i r s t']

In [34]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [35]:
[encoded] = np.array(tokenizer.texts_to_sequences([text])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [36]:
n_steps = 100
window_length = n_steps + 1
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

In [37]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [38]:
np.random.seed(42)
tf.random.set_seed(42)

In [39]:
batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [40]:
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [41]:
dataset = dataset.prefetch(1)

In [42]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 53) (32, 100)


In [56]:
#training the model, for better accuracy the number of layers and/or epochs should be increased

In [32]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=20)

Epoch 1/20
2974/2974 [==============================] - 488s 164ms/step - loss: 1.7947
Epoch 2/20
2974/2974 [==============================] - 479s 161ms/step - loss: 1.4703
Epoch 3/20
2974/2974 [==============================] - 482s 162ms/step - loss: 1.3973
Epoch 4/20
2974/2974 [==============================] - 481s 162ms/step - loss: 1.3584
Epoch 5/20
2974/2974 [==============================] - 482s 162ms/step - loss: 1.3326
Epoch 6/20
2974/2974 [==============================] - 477s 160ms/step - loss: 1.3130
Epoch 7/20
2974/2974 [==============================] - 477s 160ms/step - loss: 1.2979
Epoch 8/20
2974/2974 [==============================] - 479s 161ms/step - loss: 1.2852
Epoch 9/20
2974/2974 [==============================] - 478s 161ms/step - loss: 1.2754
Epoch 10/20
2974/2974 [==============================] - 477s 160ms/step - loss: 1.2667
Epoch 11/20
2974/2974 [==============================] - 478s 161ms/step - loss: 1.2598
Epoch 12/20
2974/2974 [==================

In [46]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)


In [58]:
#checking whether the model works, already set to the name of the loaded model (new_model), old name was 'model'

In [48]:
X_new = preprocess(["How are yo"])
Y_pred = new_model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


'u'

In [49]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

array([[0, 1, 0, 2, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 2, 1, 0, 2, 1,
        0, 1, 2, 1, 1, 1, 2, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 2]],
      dtype=int64)

In [50]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = new_model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [51]:
def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [59]:
#generating the first poem, works better with lower temperatures but the poems are less random, please feel free to manipulate \
#the temperature and comparing the outcomes \ 
#to change the length of the poem change the number in the bracket in 'tf.random.set_seed(42)'

In [52]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

the woman in the same of you 
cause i was not enough 
the same of your father 
and the same world 
ca


In [47]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.4))

to be love 
i was the rest of you 
can you call to speak up 




what does the same commanding 
and 


In [60]:
#saving the model

In [52]:
model.save('rupi kaur')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: rupi kaur\assets


In [54]:
tf.random.set_seed(20)

print(complete_text("t", temperature=0.3))

the sun 
when i was no longer than the same company 
what i am the same is a mistake 
but i was not m


In [68]:
#loading the model

In [45]:
new_model = tf.keras.models.load_model('rupi kaur')

In [63]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.4))

to be love 
i was the rest of you 
can you call to speak up 




what does the same commanding 
and 


In [65]:
tf.random.set_seed(20)

print(complete_text("t", temperature=0.5))

tention 
on the collate what i was the size of your life 
i am a mistake more than the border clothes


In [66]:
tf.random.set_seed(80)

print(complete_text("t", temperature=0.35))

the same is a good 
i want to step my father 
and her the therapist of your father 
you are a ready f


In [69]:
#the model would be improved if there were more poems to train it on, however I wanted to keep it exclusive to one author
#which meant around 500-600 poems were included
#i encourage running the model on different settings and trying your luck